In [1]:
#!pip install web3

In [2]:
#!pip install multicall

**Gearbox Protocol**

Website:        https://gearbox.fi/ <br>
dApp:           https://app.gearbox.fi/ <br>
Docs:           https://docs.gearbox.finance/ <br>
Developer Docs: https://dev.gearbox.fi/ <br>

**Gearbox Credit Accout Report** <br>
https://datastudio.google.com/reporting/a95186ae-29b4-4d72-8807-612bb5f54dd0

In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [4]:
from gearbox_ca import (pull_abi_etherscan, parse_abi, chunks, get_data_multicall, get_logs, get_token_balance) # see gearbox_ca.py
import os
from datetime import datetime
from pprint import pformat
import logging
import pandas as pd
from web3 import Web3
from web3.datastructures import AttributeDict

######### Enable logging for debug
logging.basicConfig(
        format='%(asctime)s %(funcName)s %(levelname)s %(message)s',
        level='INFO',
        datefmt='%Y-%m-%d %H:%M:%S',
        )


Etherscan_APIKEY   = None #optional, but recommended
GearboxAddressProvider = '0xcF64698AFF7E5f27A11dff868AF228653ba53be0'

# The dafault RPC from ethersjs, change it if it doesn't work: https://infura.io/docs
RPC_Endpoint = 'https://mainnet.infura.io/v3/84842078b09946638c03157f83405213'

w3_eth = Web3(Web3.HTTPProvider(RPC_Endpoint, request_kwargs={'timeout': 20}))
logging.info (f'Ethereum connected: {w3_eth.isConnected()}')


2022-05-21 12:17:35 <module> INFO Ethereum connected: True


In [5]:
logging.info('Start')
AddressProvider = Web3.toChecksumAddress(GearboxAddressProvider)
AddressProvider_abi = pull_abi_etherscan(AddressProvider)

AccountFactory = w3_eth.eth.contract(address=AddressProvider, abi=AddressProvider_abi).functions.getAccountFactory().call()
logging.info(f'AccountFactory: {AccountFactory}')
AccountFactory_abi = pull_abi_etherscan(AccountFactory)
countCreditAccounts = w3_eth.eth.contract(address=AccountFactory, abi=AccountFactory_abi).functions.countCreditAccounts().call()
countCreditAccountsInStock = w3_eth.eth.contract(address=AccountFactory, abi=AccountFactory_abi).functions.countCreditAccountsInStock().call()
logging.info(f'countCreditAccounts: {countCreditAccounts}')
logging.info(f'countCreditAccountsInStock: {countCreditAccountsInStock}')
masterCreditAccount = w3_eth.eth.contract(address=AccountFactory, abi=AccountFactory_abi).functions.masterCreditAccount().call()
logging.info(f'masterCreditAccount: {masterCreditAccount}')
CreditAccount_abi = pull_abi_etherscan(masterCreditAccount)

DataCompressor = w3_eth.eth.contract(address=AddressProvider, abi=AddressProvider_abi).functions.getDataCompressor().call()
logging.info(f'DataCompressor: {DataCompressor}')
DataCompressor_abi = pull_abi_etherscan(DataCompressor)

ContractsRegister = w3_eth.eth.contract(address=AddressProvider, abi=AddressProvider_abi).functions.getContractsRegister().call()
logging.info(f'ContractsRegister: {ContractsRegister}')
ContractsRegister_abi = pull_abi_etherscan(ContractsRegister)

CreditManagers = w3_eth.eth.contract(address=ContractsRegister, abi=ContractsRegister_abi).functions.getCreditManagers().call()
logging.info(f'CreditManagers: {CreditManagers}')

cm_dict = {AccountFactory: {'token': None, 'symbol': None, 'decimals': None}}
allowedTokens = {}
for i, CreditManager in enumerate(CreditManagers):
    logging.info(f'CreditManager {i+1} of {len(CreditManagers)}')
    if i==0:
        CreditManager_abi = pull_abi_etherscan(CreditManager)

    Token = w3_eth.eth.contract(address=CreditManager, abi=CreditManager_abi).functions.underlyingToken().call()  
    
    if i==0:
        Token_abi = pull_abi_etherscan(Token)   
    
    CreditFilter = w3_eth.eth.contract(address=CreditManager, abi=CreditManager_abi).functions.creditFilter().call()
    
    if i==0:
        CreditFilter_abi = pull_abi_etherscan(CreditFilter)
        
    allowedTokensCount = w3_eth.eth.contract(address=CreditFilter, abi=CreditFilter_abi).functions.allowedTokensCount().call()
    priceOracle        = w3_eth.eth.contract(address=CreditFilter, abi=CreditFilter_abi).functions.priceOracle().call()
    wethAddress        = w3_eth.eth.contract(address=CreditFilter, abi=CreditFilter_abi).functions.wethAddress().call()
    
    if i==0:
        priceOracle_abi = pull_abi_etherscan(priceOracle)
    
    cm_dict[CreditManager] = {'token': Token,
                              'symbol': w3_eth.eth.contract(address=Token, abi=Token_abi).functions.symbol().call(),
                              'decimals': w3_eth.eth.contract(address=Token, abi=Token_abi).functions.decimals().call(),
                              'CreditFilter': CreditFilter,
                              'priceOracle' : priceOracle,
                              'allowedTokensCount': allowedTokensCount,
                              'allowedTokens':{},
                             }
    for token_id in range(allowedTokensCount):
        allowed_token = w3_eth.eth.contract(address=CreditFilter, abi=CreditFilter_abi).functions.allowedTokens(token_id).call()
        allowed_token_symbol = w3_eth.eth.contract(address=allowed_token, abi=Token_abi).functions.symbol().call()
        allowed_token_decimals = w3_eth.eth.contract(address=allowed_token, abi=Token_abi).functions.decimals().call()
        
        try:
            allowed_token_weth_priceOracle = w3_eth.eth.contract(address=priceOracle, abi=priceOracle_abi).functions.getLastPrice(allowed_token, wethAddress).call()
        except ContractLogicError:
            allowed_token_weth_priceOracle = None
            logging.info(allowed_token_symbol+'-WETH getLastPrice error')
        
        try:    
            allowed_token_underlying_priceOracle = w3_eth.eth.contract(address=priceOracle, abi=priceOracle_abi).functions.getLastPrice(allowed_token, Token).call()
        except ContractLogicError:
            allowed_token_underlying_priceOracle = None
            logging.info(allowed_token_symbol+'-WETH getLastPrice error')

        Token_dict={'symbol'          : allowed_token_symbol, 
                    'decimals'        : allowed_token_decimals,
                    'Price_WETH'      : allowed_token_weth_priceOracle,
                    'Price_'+cm_dict[CreditManager]['symbol']: allowed_token_underlying_priceOracle,
                    }
        if allowed_token in allowedTokens:
            allowedTokens[allowed_token].update(Token_dict)
        else:
            allowedTokens[allowed_token] = Token_dict
    
df_abi = parse_abi({'AddressProvider':AddressProvider_abi, 
                    'AccountFactory': AccountFactory_abi, 
                    'DataCompressor': DataCompressor_abi,
                    'CreditManager': CreditManager_abi,
                    'CreditFilter': CreditFilter_abi,
                    'CreditAccount': CreditAccount_abi,
                   })

logging.info(pformat(cm_dict)) 
logging.info(pformat(allowedTokens))

2022-05-21 12:17:35 <module> INFO Start
2022-05-21 12:17:35 pull_abi_etherscan INFO etherscan apikey is not set: 5 sec wait due to rate-limit
2022-05-21 12:17:41 <module> INFO AccountFactory: 0x444CD42BaEdDEB707eeD823f7177b9ABcC779C04
2022-05-21 12:17:41 pull_abi_etherscan INFO etherscan apikey is not set: 5 sec wait due to rate-limit
2022-05-21 12:17:59 <module> INFO countCreditAccounts: 5001
2022-05-21 12:17:59 <module> INFO countCreditAccountsInStock: 4761
2022-05-21 12:18:00 <module> INFO masterCreditAccount: 0x373A292b93ff9017D28E64154ef83b99D5C4e270
2022-05-21 12:18:00 pull_abi_etherscan INFO etherscan apikey is not set: 5 sec wait due to rate-limit
2022-05-21 12:18:06 <module> INFO DataCompressor: 0x0050b1ABD1DD2D9b01ce954E663ff3DbCa9193B1
2022-05-21 12:18:06 pull_abi_etherscan INFO etherscan apikey is not set: 5 sec wait due to rate-limit
2022-05-21 12:18:12 <module> INFO ContractsRegister: 0xA50d4E7D8946a7c90652339CDBd262c375d54D99
2022-05-21 12:18:12 pull_abi_etherscan INFO e

In [6]:
df = pd.DataFrame()
df['id'] = range(countCreditAccounts)
for ids in list(chunks(list(df['id']), 1000)): #chunk size for multicall = 1000 (reduce in case of issues)
    id_range = list(df['id'].isin(ids))
    d_ca = get_data_multicall(w3_eth, df.loc[id_range], 'creditAccounts', df_abi, AccountFactory)
    df.loc[id_range,'CA'] = df.loc[id_range].apply(lambda x: d_ca[x['id']], axis=1)

    d_cm = get_data_multicall(w3_eth, df.loc[id_range], 'creditManager', df_abi)
    df.loc[id_range,'CM'] = df.loc[id_range].apply(lambda x: d_cm[x['id']], axis=1)
    
    d_amount = get_data_multicall(w3_eth, df.loc[id_range], 'borrowedAmount', df_abi)
    df.loc[id_range,'borrowedAmount'] = df.loc[id_range].apply(lambda x: d_amount[x['id']] if d_amount[x['id']] > 1 else 0 , axis=1)
    
    d_since = get_data_multicall(w3_eth, df.loc[id_range], 'since', df_abi)
    df.loc[id_range,'Since'] = df.loc[id_range].apply(lambda x: d_since[x['id']], axis=1)

df['Since'] = df['Since'].astype(int)
df['Decimals'] = df.apply(lambda x: cm_dict[x['CM']]['decimals'], axis=1)
df['Symbol'] = df.apply(lambda x: cm_dict[x['CM']]['symbol'] , axis=1)


In [7]:
display(df)

,id,CA,CM,borrowedAmount,Since,Decimals,Symbol
0,0,0xb5DE854F7Db3164c8F9eFeFFED4c06317BCdbBF1,0x777E23A2AcB2fCbB35f6ccF98272d03C722Ba6EB,2000000000000000000000,13858003,18.0,DAI
1,1,0xA3F1E5d5fb80B3bB0F1b04819F0930C4E0f32AF1,0x968f9a68a98819E2e6Bb910466e191A7b6cf02F0,1250000000000000000,13862947,18.0,WETH
2,2,0xe541B88d68602E5f3Fb511633bAc708BACD8EA4c,0x777E23A2AcB2fCbB35f6ccF98272d03C722Ba6EB,3000000000000000000000,13862953,18.0,DAI
3,3,0xC581Ff1a21f42B315DEDbE219a7Ed2B0fA47aBd5,0x968f9a68a98819E2e6Bb910466e191A7b6cf02F0,900000000000000000,13862987,18.0,WETH
4,4,0x5Ce9C22aC551d3b72136B3fe446902B1af0f3654,0x968f9a68a98819E2e6Bb910466e191A7b6cf02F0,854784188060446794,13864162,18.0,WETH
...,...,...,...,...,...,...,...
4996,4996,0x7E0177E914e3Cf5B69067874d4316BF552037eB2,0x444CD42BaEdDEB707eeD823f7177b9ABcC779C04,0,13819220,NaN,None
4997,4997,0xD6Cdc10FB4eCf7201e91488Ae64C4AD55e6c2515,0x444CD42BaEdDEB707eeD823f7177b9ABcC779C04,0,13819221,NaN,None
4998,4998,0xD1168F931863CCcEf0102041f275cBA5C49907a2,0x444CD42BaEdDEB707eeD823f7177b9ABcC779C04,0,13819222,NaN,None
4999,4999,0xee6B3a0B5a750902c4513508bef90963e88F1c80,0x444CD42BaEdDEB707eeD823f7177b9ABcC779C04,0,13819222,NaN,None


In [8]:
#Open, Close, Repay, Liquidate
OpenCreditAccount_topic      =  df_abi[(df_abi['name']=='OpenCreditAccount') &(df_abi['type']=='event')]['topic'].values[0]
AddCollateral_topic          =  df_abi[(df_abi['name']=='AddCollateral')&(df_abi['type']=='event')]['topic'].values[0]
CloseCreditAccount_topic     =  df_abi[(df_abi['name']=='CloseCreditAccount')&(df_abi['type']=='event')]['topic'].values[0]
RepayCreditAccount_topic     =  df_abi[(df_abi['name']=='RepayCreditAccount')&(df_abi['type']=='event')]['topic'].values[0]
LiquidateCreditAccount_topic =  df_abi[(df_abi['name']=='LiquidateCreditAccount')&(df_abi['type']=='event')]['topic'].values[0]

logging.info(f'OpenCreditAccount_topic: {OpenCreditAccount_topic}' )
logging.info(f'CloseCreditAccount_topic: {CloseCreditAccount_topic}')
logging.info(f'RepayCreditAccount_topic: {RepayCreditAccount_topic}')
logging.info(f'LiquidateCreditAccount_topic: {LiquidateCreditAccount_topic}')

logs = pd.DataFrame()
for CM in CreditManagers:
    CM_logs = get_logs(w3_eth, CM, df_abi,
                       [[OpenCreditAccount_topic,
                         AddCollateral_topic,
                         CloseCreditAccount_topic,
                         RepayCreditAccount_topic,
                         LiquidateCreditAccount_topic]
                         ],
                         df.loc[df['CM']==CM]['Since'].min(),
                         'latest')
    logs = logs.append(CM_logs, ignore_index = True) 

i=0
for row in df.loc[df['CM'].isin(CreditManagers)].itertuples():
    i+=1
    open_events       = logs[(logs['address']==row.CM) & (logs['blockNumber']>=row.Since) & (logs['event']=='OpenCreditAccount')]['args'].values 
    collateral_events = logs[(logs['address']==row.CM) & (logs['blockNumber']>=row.Since) & (logs['event']=='AddCollateral')]['args'].values 
    close_events      = logs[(logs['address']==row.CM) & (logs['blockNumber']>=row.Since) & (logs['event']=='CloseCreditAccount')]['args'].values 
    repay_events      = logs[(logs['address']==row.CM) & (logs['blockNumber']>=row.Since) & (logs['event']=='RepayCreditAccount')]['args'].values
    liquidate_event   = logs[(logs['address']==row.CM) & (logs['blockNumber']>=row.Since) & (logs['event']=='LiquidateCreditAccount')]['args'].values
    
    CA_open_event = [x for x in open_events if x['creditAccount']== row.CA] # Open
    if len(CA_open_event) > 0:
        Borrower = CA_open_event[0]['onBehalfOf']
        df.loc[df['id']==row.id, 'Borrower'] = Borrower
        
        Collateral = sum([x['amount'] for x in CA_open_event])
        CA_collateral_event = [x for x in collateral_events if x['onBehalfOf']== Borrower] # Open
        if len(CA_collateral_event) > 0:
            #Collateral = Collateral + sum([x['value'] for x in CA_collateral_event])
            Collateral = Collateral + sum(
                                            [x['value']*(allowedTokens[x['token']]['Price_'+row.Symbol])*(10**-allowedTokens[x['token']]['decimals'])
                                           /(allowedTokens[cm_dict[row.CM]['token']]['Price_'+row.Symbol]*(10**-allowedTokens[cm_dict[row.CM]['token']]['decimals']))
                                           for x in CA_collateral_event]
                                         )
        df.loc[df['id']==row.id, 'Collateral'] = Collateral
                           
        CA_close_event = [x for x in close_events if x['to']== Borrower] # Close
        if len(CA_close_event) > 0:
            df.loc[df['id']==row.id, ['Borrower', 'borrowedAmount', 'Collateral']] = [None, 0, 0]

        CA_repay_event = [x for x in repay_events if x['to']== Borrower] # Repay
        if len(CA_repay_event) > 0:
            df.loc[df['id']==row.id, ['Borrower', 'borrowedAmount', 'Collateral']] = [None, 0, 0]

        CA_liquidate_event = [x for x in liquidate_event if x['owner']== Borrower] # Liquidate
        if len(CA_liquidate_event) > 0:
            df.loc[df['id']==row.id, ['Borrower', 'borrowedAmount', 'Collateral']] = [None, 0, 0]
            
    if i % 50 == 0:
        logging.info (i)

logging.info(f'{i} end')


2022-05-21 12:21:16 <module> INFO OpenCreditAccount_topic: 0x7b20ae77867a263a1074203a2da261ef0d096c99395c59c9d4a0104b9f334a27
2022-05-21 12:21:16 <module> INFO CloseCreditAccount_topic: 0xca05b632388199c23de1352b2e96fd72a0ec71611683330b38060c004bbf0a76
2022-05-21 12:21:16 <module> INFO RepayCreditAccount_topic: 0xe7c7987373a0cc4913d307f23ab8ef02e0333a2af445065e2ef7636cffc6daa7
2022-05-21 12:21:16 <module> INFO LiquidateCreditAccount_topic: 0x5e5da6c348e62989f9cfe029252433fc99009b7d28fa3c20d675520a10ff5896
2022-05-21 12:21:19 <module> INFO 50
2022-05-21 12:21:19 <module> INFO 100
2022-05-21 12:21:20 <module> INFO 150
2022-05-21 12:21:20 <module> INFO 200
2022-05-21 12:21:20 <module> INFO 250
2022-05-21 12:21:21 <module> INFO 300
2022-05-21 12:21:21 <module> INFO 350
2022-05-21 12:21:21 <module> INFO 400
2022-05-21 12:21:22 <module> INFO 450
2022-05-21 12:21:22 <module> INFO 500
2022-05-21 12:21:22 <module> INFO 550
2022-05-21 12:21:23 <module> INFO 600
2022-05-21 12:21:23 <module> INFO 

In [9]:
display(df)

,id,CA,CM,borrowedAmount,Since,Decimals,Symbol,Borrower,Collateral
0,0,0xb5DE854F7Db3164c8F9eFeFFED4c06317BCdbBF1,0x777E23A2AcB2fCbB35f6ccF98272d03C722Ba6EB,0,13858003,18.0,DAI,None,0.000000e+00
1,1,0xA3F1E5d5fb80B3bB0F1b04819F0930C4E0f32AF1,0x968f9a68a98819E2e6Bb910466e191A7b6cf02F0,0,13862947,18.0,WETH,None,0.000000e+00
2,2,0xe541B88d68602E5f3Fb511633bAc708BACD8EA4c,0x777E23A2AcB2fCbB35f6ccF98272d03C722Ba6EB,0,13862953,18.0,DAI,None,0.000000e+00
3,3,0xC581Ff1a21f42B315DEDbE219a7Ed2B0fA47aBd5,0x968f9a68a98819E2e6Bb910466e191A7b6cf02F0,0,13862987,18.0,WETH,None,0.000000e+00
4,4,0x5Ce9C22aC551d3b72136B3fe446902B1af0f3654,0x968f9a68a98819E2e6Bb910466e191A7b6cf02F0,854784188060446794,13864162,18.0,WETH,0x53829d517D8fA7D59d3D40E20251e519C079985F,3.000000e+17
...,...,...,...,...,...,...,...,...,...
4996,4996,0x7E0177E914e3Cf5B69067874d4316BF552037eB2,0x444CD42BaEdDEB707eeD823f7177b9ABcC779C04,0,13819220,NaN,None,NaN,NaN
4997,4997,0xD6Cdc10FB4eCf7201e91488Ae64C4AD55e6c2515,0x444CD42BaEdDEB707eeD823f7177b9ABcC779C04,0,13819221,NaN,None,NaN,NaN
4998,4998,0xD1168F931863CCcEf0102041f275cBA5C49907a2,0x444CD42BaEdDEB707eeD823f7177b9ABcC779C04,0,13819222,NaN,None,NaN,NaN
4999,4999,0xee6B3a0B5a750902c4513508bef90963e88F1c80,0x444CD42BaEdDEB707eeD823f7177b9ABcC779C04,0,13819222,NaN,None,NaN,NaN


In [10]:

data_cols = [x['name'] for x in df_abi[df_abi['name']=='getCreditAccountDataExtended']['abi'].values[0]['outputs'][0]['components']]

batchtime = datetime.utcnow()
df['batchtime'] = batchtime

for ids in list(chunks(list(df[pd.notna(df['Borrower'])].loc[:,'id']), 1000)): #chunk size for multicall = 1000 (reduce in case of issues)
    id_range = list(df['id'].isin(ids))
    try:
        d_data = get_data_multicall(w3_eth, df.loc[id_range], 'getCreditAccountDataExtended', df_abi, DataCompressor)
    except ContractLogicError:
        logging.error('getCreditAccountDataExtended: Multicall error, calling it one by one')
        d_data = {x:getCreditAccountDataExtended(x,y,z) for x,y,z in zip(df.loc[id_range]['id'],df.loc[id_range]['CM'],df.loc[id_range]['Borrower'])}
        d_data = AttributeDict.recursive(d_data)
    for data in data_cols:
        if data not in ['balances', 'inUse', 'borrower', 'addr', 'borrower', 'creditManager', 'since']: #duplicate columns
            df.loc[id_range, data] = df.loc[id_range].apply(lambda x: {y:z for y, z in zip(data_cols, d_data[x['id']])}[data] if d_data[x['id']] else None
                                                            , axis=1)
        
    tokens_to_frame = [allowedTokens[x]['symbol'] for x in allowedTokens] 
    for token in tokens_to_frame:
        df.loc[id_range, 'Balance_'+token] = df.loc[id_range].apply(lambda x: get_token_balance(x, token, data_cols, d_data, allowedTokens)
                                                        , axis=1)
    


In [11]:
display(df) 

,id,CA,CM,borrowedAmount,Since,Decimals,Symbol,Borrower,Collateral,batchtime,...,Balance_WBTC,Balance_WETH,Balance_YFI,Balance_yvDAI,Balance_yvUSDC,Balance_CRV,Balance_SUSHI,Balance_LDO,Balance_FTM,Balance_LUNA
0,0,0xb5DE854F7Db3164c8F9eFeFFED4c06317BCdbBF1,0x777E23A2AcB2fCbB35f6ccF98272d03C722Ba6EB,0,13858003,18.0,DAI,None,0.000000e+00,2022-05-21 00:21:23.584949,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,0xA3F1E5d5fb80B3bB0F1b04819F0930C4E0f32AF1,0x968f9a68a98819E2e6Bb910466e191A7b6cf02F0,0,13862947,18.0,WETH,None,0.000000e+00,2022-05-21 00:21:23.584949,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,0xe541B88d68602E5f3Fb511633bAc708BACD8EA4c,0x777E23A2AcB2fCbB35f6ccF98272d03C722Ba6EB,0,13862953,18.0,DAI,None,0.000000e+00,2022-05-21 00:21:23.584949,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,0xC581Ff1a21f42B315DEDbE219a7Ed2B0fA47aBd5,0x968f9a68a98819E2e6Bb910466e191A7b6cf02F0,0,13862987,18.0,WETH,None,0.000000e+00,2022-05-21 00:21:23.584949,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,0x5Ce9C22aC551d3b72136B3fe446902B1af0f3654,0x968f9a68a98819E2e6Bb910466e191A7b6cf02F0,854784188060446794,13864162,18.0,WETH,0x53829d517D8fA7D59d3D40E20251e519C079985F,3.000000e+17,2022-05-21 00:21:23.584949,...,0.0,1154784188060446794,0.0,0,0.0,0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4996,4996,0x7E0177E914e3Cf5B69067874d4316BF552037eB2,0x444CD42BaEdDEB707eeD823f7177b9ABcC779C04,0,13819220,NaN,None,None,NaN,2022-05-21 00:21:23.584949,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4997,4997,0xD6Cdc10FB4eCf7201e91488Ae64C4AD55e6c2515,0x444CD42BaEdDEB707eeD823f7177b9ABcC779C04,0,13819221,NaN,None,None,NaN,2022-05-21 00:21:23.584949,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4998,4998,0xD1168F931863CCcEf0102041f275cBA5C49907a2,0x444CD42BaEdDEB707eeD823f7177b9ABcC779C04,0,13819222,NaN,None,None,NaN,2022-05-21 00:21:23.584949,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4999,4999,0xee6B3a0B5a750902c4513508bef90963e88F1c80,0x444CD42BaEdDEB707eeD823f7177b9ABcC779C04,0,13819222,NaN,None,None,NaN,2022-05-21 00:21:23.584949,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
display(df[pd.notna(df['Borrower'])]) # active CAs

,id,CA,CM,borrowedAmount,Since,Decimals,Symbol,Borrower,Collateral,batchtime,...,Balance_WBTC,Balance_WETH,Balance_YFI,Balance_yvDAI,Balance_yvUSDC,Balance_CRV,Balance_SUSHI,Balance_LDO,Balance_FTM,Balance_LUNA
4,4,0x5Ce9C22aC551d3b72136B3fe446902B1af0f3654,0x968f9a68a98819E2e6Bb910466e191A7b6cf02F0,854784188060446794,13864162,18.0,WETH,0x53829d517D8fA7D59d3D40E20251e519C079985F,3.000000e+17,2022-05-21 00:21:23.584949,...,0.0,1154784188060446794,0.0,0,0.000000e+00,0,0.0,0.0,0.0,0
8,8,0xF91027C04807c71c7771a2314A622dFA0fE509A1,0x968f9a68a98819E2e6Bb910466e191A7b6cf02F0,1000000000000000000,13864789,18.0,WETH,0x896b94f4f27f12369698C302e2049cAe86936BbB,1.000000e+18,2022-05-21 00:21:23.584949,...,0.0,2000000000000000000,0.0,0,0.000000e+00,0,0.0,0.0,0.0,0
11,11,0x0B9851B7bcE408dd36d3473a0A0eE8F14ce64ca3,0x968f9a68a98819E2e6Bb910466e191A7b6cf02F0,290000000000000000,13864924,18.0,WETH,0xC16414AC1fedfDAC4F8A09674D994e1BbB9d7113,1.000000e+18,2022-05-21 00:21:23.584949,...,0.0,1290000000000000000,0.0,0,0.000000e+00,0,0.0,0.0,0.0,0
14,14,0x654C0aDcA189042B16A849d2fF2ED476F525c39e,0x777E23A2AcB2fCbB35f6ccF98272d03C722Ba6EB,15000000000000000000000,13865587,18.0,DAI,0x3aB0818Ba16175D5a41a4076De227c8a55AD6038,1.000000e+22,2022-05-21 00:21:23.584949,...,0.0,0,0.0,0,0.000000e+00,0,0.0,0.0,0.0,0
20,20,0xffaDbA97754864Ea63956026836E317e918Ce26d,0x777E23A2AcB2fCbB35f6ccF98272d03C722Ba6EB,1500000000000000000000,13867574,18.0,DAI,0x5299227765598896bd240220CE1810d7Cf126bEc,1.500000e+21,2022-05-21 00:21:23.584949,...,0.0,1298654873911843983,0.0,0,0.000000e+00,0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
675,675,0xa8929716aDb9C518271EE4D9B5D5369797C11b7b,0x2664cc24CBAd28749B3Dd6fC97A6B402484De527,79997724951,14795678,6.0,USDC,0xE3939654Deae5f54fD3e6B84b3A7F75f245062d8,5.000348e+10,2022-05-21 00:21:23.584949,...,0.0,0,0.0,0,1.280067e+11,0,0.0,0.0,0.0,0
676,676,0x7A56bAf4eB568E870d29326b4D30db37B8d82b86,0x968f9a68a98819E2e6Bb910466e191A7b6cf02F0,900000000000000000,14795825,18.0,WETH,0xA08A327D9FB9f2be150Dd7b7c20f077d3A997FEB,3.000000e+17,2022-05-21 00:21:23.584949,...,2051780.0,900000000000000000,0.0,0,0.000000e+00,0,0.0,0.0,0.0,0
677,677,0xD2257354aA72619ab3c91341E541B508Cf3b24A4,0x2664cc24CBAd28749B3Dd6fC97A6B402484De527,3000000000,14797367,6.0,USDC,0xB0404a7B9E67EFA301488F26C3F3F9D091a92bC9,1.000000e+09,2022-05-21 00:21:23.584949,...,0.0,0,0.0,0,0.000000e+00,0,0.0,0.0,0.0,0
678,678,0x86e19CdAda876A44881878053e5E6A3A06bDE97F,0x2664cc24CBAd28749B3Dd6fC97A6B402484De527,3000000000,14801823,6.0,USDC,0xc7aC422595Da7c1aef9e2f1119a015272C27faF7,1.000000e+09,2022-05-21 00:21:23.584949,...,0.0,0,0.0,0,0.000000e+00,0,0.0,0.0,0.0,0


In [13]:
logging.info(f'batchtime {batchtime}' )
logging.info(f'countCreditAccounts - countCreditAccountsInStock = {countCreditAccounts - countCreditAccountsInStock}')

2022-05-21 12:21:33 <module> INFO batchtime 2022-05-21 00:21:23.584949
2022-05-21 12:21:33 <module> INFO countCreditAccounts - countCreditAccountsInStock = 240


In [14]:
df_price_oracle = pd.DataFrame.from_dict([allowedTokens[x] for x in allowedTokens])
df_price_oracle = df_price_oracle.set_index('symbol', drop=True).transpose()
df_price_oracle.columns = ['Price_'+x for x in df_price_oracle.columns]
df_price_oracle[[x.replace('Price','Decimals') for x in df_price_oracle.columns]] = df_price_oracle.loc['decimals',:]
df_price_oracle = df_price_oracle.drop(index = 'decimals')

df_price_oracle = df_price_oracle.reset_index()
df_price_oracle = df_price_oracle.rename(columns={'index': 'Price_Asset'})
df_price_oracle['Price_Asset'] = df_price_oracle['Price_Asset'].apply(lambda x: x.replace('Price_',''))

df_price_oracle['batchtime'] = batchtime


In [15]:
display(df_price_oracle)

,Price_Asset,Price_DAI,Price_1INCH,Price_AAVE,Price_COMP,Price_DPI,Price_FEI,Price_LINK,Price_SNX,Price_UNI,...,Decimals_WETH,Decimals_YFI,Decimals_yvDAI,Decimals_yvUSDC,Decimals_CRV,Decimals_SUSHI,Decimals_LDO,Decimals_FTM,Decimals_LUNA,batchtime
0,WETH,509517820382821,465194967694129,46200000000000000,33390045826867484,47903550031084690,516320000000000,3499017571185742,1303473312401884,2564000000000000,...,18,18,18,6,18,18,18,18,18,2022-05-21 00:21:23.584949
1,DAI,1000000000000000000,913010201183168670,90673963013282053226,65532635937601190823,94017418262412976887,1013350229069649128,6867311468236363070,2558248721158629282,5032208683247947715,...,18,18,18,6,18,18,18,18,18,2022-05-21 00:21:23.584949
2,USDC,1004989109909963611,917565309425789487,91126345380727292664,65859585460983514412,94486481495575175270,1018405944739764410,6901573239937347866,2571012105205513535,5057314925458544986,...,18,18,18,6,18,18,18,18,18,2022-05-21 00:21:23.584949
3,WBTC,34244415897997,31265501048430,3105076900546457,2244126839939261,3219571572634039,34701586694591,235167070019372,87605733177875,172325047034656,...,18,18,18,6,18,18,18,18,18,2022-05-21 00:21:23.584949


In [31]:
df_token_price = pd.DataFrame.from_dict([allowedTokens[x] for x in allowedTokens])
df_token_price.columns = [x.replace('Price_','') for x in df_token_price.columns]
df_token_price['batchtime'] = batchtime
df_token_price = df_token_price.rename(columns = {'symbol':'token'})
numeric_cols = [x for x in df_token_price.columns if x not in ['token', 'batchtime','decimals']]
df_token_price[numeric_cols] = df_token_price[numeric_cols]/1e18
df_token_price[numeric_cols] = df_token_price[numeric_cols].astype('float64')

In [33]:
display(df_token_price)

,token,decimals,WETH,DAI,USDC,WBTC,batchtime
0,DAI,18,0.000510,1.000000,1.004989,0.000034,2022-05-21 00:21:23.584949
1,1INCH,18,0.000465,0.913010,0.917565,0.000031,2022-05-21 00:21:23.584949
2,AAVE,18,0.046200,90.673963,91.126345,0.003105,2022-05-21 00:21:23.584949
3,COMP,18,0.033390,65.532636,65.859585,0.002244,2022-05-21 00:21:23.584949
4,DPI,18,0.047904,94.017418,94.486481,0.003220,2022-05-21 00:21:23.584949
5,FEI,18,0.000516,1.013350,1.018406,0.000035,2022-05-21 00:21:23.584949
6,LINK,18,0.003499,6.867311,6.901573,0.000235,2022-05-21 00:21:23.584949
7,SNX,18,0.001303,2.558249,2.571012,0.000088,2022-05-21 00:21:23.584949
8,UNI,18,0.002564,5.032209,5.057315,0.000172,2022-05-21 00:21:23.584949
9,USDC,6,0.000507,0.995036,1.000000,0.000034,2022-05-21 00:21:23.584949


In [18]:
#Open, Close, Repay, Liquidate, AddCollateral, IncreaseBorrowedAmount
OpenCreditAccount_topic      =  df_abi[(df_abi['name']=='OpenCreditAccount') &(df_abi['type']=='event')]['topic'].values[0]
CloseCreditAccount_topic     =  df_abi[(df_abi['name']=='CloseCreditAccount')&(df_abi['type']=='event')]['topic'].values[0]
RepayCreditAccount_topic     =  df_abi[(df_abi['name']=='RepayCreditAccount')&(df_abi['type']=='event')]['topic'].values[0]
LiquidateCreditAccount_topic =  df_abi[(df_abi['name']=='LiquidateCreditAccount')&(df_abi['type']=='event')]['topic'].values[0]
AddCollateral_topic          =  df_abi[(df_abi['name']=='AddCollateral')&(df_abi['type']=='event')]['topic'].values[0]
IncreaseBorrowedAmount_topic =  df_abi[(df_abi['name']=='IncreaseBorrowedAmount')&(df_abi['type']=='event')]['topic'].values[0]

from_block = 0

logging.info(f'from_block={from_block}')

df_events = pd.DataFrame()

for CM in CreditManagers:
    logs = get_logs(w3_eth, CM, df_abi, 
                    [[OpenCreditAccount_topic, 
                      CloseCreditAccount_topic, 
                      RepayCreditAccount_topic, 
                      LiquidateCreditAccount_topic,
                      AddCollateral_topic,
                      IncreaseBorrowedAmount_topic,]
                    ],
                    from_block,
                    'latest')
    logging.info(f'get_logs from CM {CM}')
    df_events = df_events.append(logs, ignore_index = True)    

logging.info(f'number of events={len(df_events)}')
if len(df_events)>0:
    df_events = df_events.rename(columns={'address': 'CM'})
    df_events['CM_Token'] = df_events['CM'].apply(lambda x: cm_dict[x]['symbol'])
    df_events['blockHash'] = df_events['blockHash'].apply(lambda x: x.hex())
    df_events['transactionHash'] = df_events['transactionHash'].apply(lambda x: x.hex())
    df_events['Borrower'] = df_events.apply(lambda x: x['args']['to'] 
                                                  if x['event'] in ['CloseCreditAccount','CloseCreditAccount'] 
                                                  else x['args']['onBehalfOf'] if x['event'] in ['OpenCreditAccount','AddCollateral']
                                                  else x['args']['borrower'] if x['event'] in ['IncreaseBorrowedAmount']
                                                  else x['args']['owner'] if x['event'] in ['LiquidateCreditAccount']
                                                  else None
                                        ,axis=1)
    df_events['Liquidator'] = df_events.apply(lambda x: x['args']['liquidator'] if x['event'] in ['LiquidateCreditAccount']
                                                  else None
                                        ,axis=1)
    df_events['Collateral_Token'] = df_events.apply(lambda x: allowedTokens[x['args']['token']]['symbol'] if x['event']=='AddCollateral'
                                                    else cm_dict[x['CM']]['symbol']
                                        ,axis=1)
    df_events['Collateral'] = df_events.apply(lambda x: x['args']['amount']/(10**cm_dict[x['CM']]['decimals']) if x['event']=='OpenCreditAccount'
                                                    else x['args']['value']/(10**allowedTokens[x['args']['token']]['decimals']) if x['event']=='AddCollateral'
                                                    else None
                                        ,axis=1)

    logging.info('pull event timestamp begin')
    df_events['blockTimestamp'] = df_events['blockNumber'].apply(lambda x: datetime.utcfromtimestamp(w3_eth.eth.getBlock(x)['timestamp']))
    logging.info('pull event timestamp end')
    
    logging.info('pull transaction fee begin')
    df_events['transaction_receipt'] = df_events['transactionHash'].apply(lambda x: dict(w3_eth.eth.get_transaction_receipt(x)))
    df_events['transaction_fee'] = df_events['transaction_receipt'].apply(lambda x: x['gasUsed']*x['effectiveGasPrice']/1e18)
    logging.info('pull transaction fee end')    
    
    df_events[['args','transaction_receipt']] = df_events[['args','transaction_receipt']].apply(str)
    df_events['Liquidator'] = df_events['Liquidator'].astype('object')    
    df_events['Collateral'] = df_events['Collateral'].astype('float64')
    df_events.drop(columns = 'logIndex')
    df_events = df_events.sort_values('blockTimestamp', ignore_index = True)

else:
    logging.info(f'No new events since block {from_block}')
    

2022-05-21 12:21:33 <module> INFO from_block=0
2022-05-21 12:21:34 <module> INFO get_logs from CM 0x777E23A2AcB2fCbB35f6ccF98272d03C722Ba6EB
2022-05-21 12:21:35 <module> INFO get_logs from CM 0x2664cc24CBAd28749B3Dd6fC97A6B402484De527
2022-05-21 12:21:36 <module> INFO get_logs from CM 0x968f9a68a98819E2e6Bb910466e191A7b6cf02F0
2022-05-21 12:21:36 <module> INFO get_logs from CM 0xC38478B0A4bAFE964C3526EEFF534d70E1E09017
2022-05-21 12:21:36 <module> INFO number of events=1254
2022-05-21 12:21:36 <module> INFO pull event timestamp begin
2022-05-21 12:26:33 <module> INFO pull event timestamp end
2022-05-21 12:26:33 <module> INFO pull transaction fee begin
2022-05-21 12:31:38 <module> INFO pull transaction fee end


In [19]:
display(df_events)

,CM,args,blockHash,blockNumber,event,logIndex,transactionHash,transactionIndex,CM_Token,Borrower,Liquidator,Collateral_Token,Collateral,blockTimestamp,transaction_receipt,transaction_fee
0,0x777E23A2AcB2fCbB35f6ccF98272d03C722Ba6EB,"0 (sender, onBehalfOf, creditAccount, am...",0x6191b1dc0938794510eb96bc4e9b1ec8d1a91fa9352b...,13858003,OpenCreditAccount,316,0xc322fdd8d731a58b3e95414d3201fa53be82fd780e82...,220,DAI,0x8a4B89D76A1a745a4A1aDEBd3793253FBa0adADc,None,DAI,1000.00,2021-12-22 23:40:25,0 {'blockHash': b'a\x91\xb1\xdc\t8yE\x10...,0.029386
1,0x968f9a68a98819E2e6Bb910466e191A7b6cf02F0,"0 (sender, onBehalfOf, creditAccount, am...",0x880436c3d9131fc9a17a6d79ad2acdcd50974d1cde97...,13862947,OpenCreditAccount,458,0x37aa66296c27993bdc5e81e27c6d819e7466df0c070c...,356,WETH,0x9Fb7C29a87a2C5Eaf01ED3f0f4Eff040a81c9eCA,None,WETH,1.25,2021-12-23 18:09:38,0 {'blockHash': b'a\x91\xb1\xdc\t8yE\x10...,0.047534
2,0x777E23A2AcB2fCbB35f6ccF98272d03C722Ba6EB,"0 (sender, onBehalfOf, creditAccount, am...",0xef59810b1015fa4f1a64d68a6166fe9d23e40cb1f585...,13862953,OpenCreditAccount,168,0xae23bb735281dbfede5cc3352dad8b7bdaa529b27989...,125,DAI,0x7BAFC0D5c5892f2041FD9F2415A7611042218e22,None,DAI,1000.00,2021-12-23 18:11:05,0 {'blockHash': b'a\x91\xb1\xdc\t8yE\x10...,0.036783
3,0x968f9a68a98819E2e6Bb910466e191A7b6cf02F0,"0 (sender, onBehalfOf, creditAccount, am...",0x60a3c671e4f5d06281611a576410de04c4ec1478971b...,13862954,AddCollateral,434,0xf37a9984138c6b9c02bc657593ef6d0680dd69e2f002...,312,WETH,0x9Fb7C29a87a2C5Eaf01ED3f0f4Eff040a81c9eCA,None,WETH,0.55,2021-12-23 18:11:07,0 {'blockHash': b'a\x91\xb1\xdc\t8yE\x10...,0.011669
4,0x968f9a68a98819E2e6Bb910466e191A7b6cf02F0,"0 (sender, onBehalfOf, creditAccount, am...",0x6db00f3e2855ea71eb99701dfd5adb413666e13b2e72...,13862971,CloseCreditAccount,300,0x380f4f2acff142b5a407b51cabbd50b611d9702aa99b...,160,WETH,0x9Fb7C29a87a2C5Eaf01ED3f0f4Eff040a81c9eCA,None,WETH,NaN,2021-12-23 18:14:12,0 {'blockHash': b'a\x91\xb1\xdc\t8yE\x10...,0.029957
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1249,0x968f9a68a98819E2e6Bb910466e191A7b6cf02F0,"0 (sender, onBehalfOf, creditAccount, am...",0xc706dc0d4ed3ea1eeaf834866410da1a328db72236ea...,14810052,IncreaseBorrowedAmount,390,0x9c2bb2cc421e533aa4c78c056123a33bb7638507b2e4...,256,WETH,0x2ce1A66F22A2Dc6E410D9021D57aEB8D13D6bFEF,None,WETH,NaN,2022-05-20 08:32:53,0 {'blockHash': b'a\x91\xb1\xdc\t8yE\x10...,0.003085
1250,0x2664cc24CBAd28749B3Dd6fC97A6B402484De527,"0 (sender, onBehalfOf, creditAccount, am...",0xe7b93240455743061b09ff4ecff916c5799eb9c6e0f7...,14810543,CloseCreditAccount,279,0x2f65e2ded14356eb0479378289ad3939d93792d419ad...,175,USDC,0x360f9f42411b3355BC448a28F1b399f2718a9363,None,USDC,NaN,2022-05-20 10:21:50,0 {'blockHash': b'a\x91\xb1\xdc\t8yE\x10...,0.004135
1251,0x2664cc24CBAd28749B3Dd6fC97A6B402484De527,"0 (sender, onBehalfOf, creditAccount, am...",0x9797198881d865957110a2c4a8ae89c8e0af8ec24134...,14811743,CloseCreditAccount,393,0xc9a2bb8d0943f6cd111b18a76f997aa20adcd8aa1982...,202,USDC,0x0206171CdD913Fc2df3d1B084291F424Ef680d94,None,USDC,NaN,2022-05-20 14:56:14,0 {'blockHash': b'a\x91\xb1\xdc\t8yE\x10...,0.018988
1252,0x968f9a68a98819E2e6Bb910466e191A7b6cf02F0,"0 (sender, onBehalfOf, creditAccount, am...",0x47d1b8fe8ce114e7ffafc6486e2df3f6e1ccf8903ee6...,14812265,OpenCreditAccount,939,0x5a037e364c87bd4c19c8f46229f8ee71e584dab4a827...,252,WETH,0xfA2982Ba84F578E15Aec495A03556c8e8A638FBa,None,WETH,0.38,2022-05-20 16:56:32,0 {'blockHash': b'a\x91\xb1\xdc\t8yE\x10...,0.015775


# Data Studio Report
https://datastudio.google.com/reporting/a95186ae-29b4-4d72-8807-612bb5f54dd0